In [ ]:
from pyspark.sql import SparkSession

In [ ]:
class LazySparkSession:
    packages = [
        "io.delta:delta-spark_2.13:4.0.0",
        "org.apache.hadoop:hadoop-aws:3.3.4",
        "com.amazonaws:aws-java-sdk-bundle:1.12.262",
        # "io.unitycatalog:unitycatalog-spark_2.12:0.2.0",
        # "org.apache.hadoop:hadoop-common:3.3.4",
    ]

    def __init__(self, access_key, secret_key, endpoint):
        self._access_key = access_key
        self._secret_key = secret_key
        self._endpoint = endpoint
        

    def start(
        self,
        app_name: str = "Airflow Spark Delta Minio App",
        executor_memory: str = "1g",
        driver_memory: str = "1g",
        driver_maxresultsize: str = "1g",
        master_url: str = "local[*]",
    ):

        builder = (
            SparkSession
            .Builder()
            .appName(app_name)
            #
            .config("spark.executor.memory", executor_memory)
            .config("spark.driver.memory", driver_memory)
            .config("spark.driver.maxResultSize", driver_maxresultsize)
            #
            .config("spark.jars.packages", ",".join(self.packages))
            #
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
            #
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
            #
            .config("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
            .config("spark.hadoop.fs.s3a.access.key", self._access_key)
            .config("spark.hadoop.fs.s3a.secret.key", self._secret_key)
            .config("spark.hadoop.fs.s3a.endpoint", self._endpoint)
            #
            .config("spark.hadoop.delta.enableFastS3AListFrom", "true")
            #
            # .config("spark.sql.catalog.spark_catalog", "io.unitycatalog.spark.UCSingleCatalog")
            # .config("spark.sql.catalog.unity", "io.unitycatalog.spark.UCSingleCatalog")
            # .config("spark.sql.catalog.unity.uri", "http://127.0.0.1:8080")
            # .config("spark.sql.catalog.unity.token", "")
            # .config("spark.sql.defaultCatalog", "unity")
            .master(master_url)
        )

        return builder.getOrCreate()

In [ ]:
spark = LazySparkSession(access_key="LE0IecL2Ab6FdW1zIcIk", secret_key="f4AkG8krYzjaUJLzDgVFwCTkPAD9p6VeUNFcID0H", endpoint="http://127.0.0.1:10001").start()